<a id='Topic0'></a>
# Analyzing the Impact of Climate Change on U.S. Crop Yields

In this notebook, we explore the relationship between climate change and crop yields in the United States. More specifically, in this notebook, we will:

1.  Extract, clean, and preprocess crop yield data sourced from the agricultural survey conducted by the National Agricultural Statistics Service (NASS), a division of the United States Department of Agriculture (USDA).

2. Extract, clean, and preprocess climate data sourced from weather stations operated by the National Oceanic and Atmospheric Administration (NOAA). These weather stations capture daily records of several climate variables. For this analysis, we are will only consider maximum, minimum, and average temperature, along with precipitation.

3. Merge the two datasets and visualize the data to uncover potential trends or insights that can inform our understanding of how temperature variations impact agricultural productivity. 


<a href='#toc'>TOC</a>

In [16]:
# Import libraries
import pandas as pd
import urllib.parse
import altair as alt

### Extract, Clean, and Preprocess Crop Yield Data from the USDA Database

The crop yield data will be retrieved from the National Agricultural Statistics Service (NASS) QuickStats database. This database will be programmatically accessed using the QuickStats API Key. For this analysis, we will focus on retrieving the yield data and metadata for the top 4 crops grown in the US. We will further narrow our scope to include only the states that are the largest producers of the crops of interest:

1. Corn grown in Iowa
2. Soybean grown in Illinois
3. Barley grown in Idaho
4. Oats grown in South Dakota

In [3]:
# Use the QuickStats API key to access the database and retrieve and return the crop yield data
def get_data(parameters):

    # Base URL which contains the QuickStats API Key
    base_url = 'http://quickstats.nass.usda.gov/api/api_GET/?key=D48CBA42-D6F6-31A1-9EF1-46D1A70988F5&' 
    full_url = base_url + parameters
            
    df = pd.read_csv(full_url)
            
    return df  

In [4]:
# Define the filter_list variable according to our crop selection criteria that we will use for our analysis
filter_list = [
    ('IOWA','CORN','YIELD','CORN, GRAIN - YIELD, MEASURED IN BU / ACRE'),
    ('ILLINOIS','SOYBEANS','YIELD','SOYBEANS - YIELD, MEASURED IN BU / ACRE'),
    ('IDAHO','BARLEY','YIELD','BARLEY - YIELD, MEASURED IN BU / ACRE'),
    ('SOUTH DAKOTA','OATS','YIELD','OATS - YIELD, MEASURED IN BU / ACRE')
    ]

crop_yield_df = pd.DataFrame()

# Iterate through filter_list to create the final crop_yield_df dataframe that will be used for our analysis
# To use the API key, small snippets of the code below were inspired from a script shared in the following post: https://www.reddit.com/r/learnpython/comments/1dz8v75/trying_to_access_usda_nass_api_with_no_success/
for state,commodity,stat_desc,description in filter_list:

    # Set up filter criteria that will be used to access the database and retireve the yield dataSS
    source_desc = 'source_desc=SURVEY'
    sector_desc = '&sector_desc=CROPS'
    commodity_desc = f'&commodity_desc={commodity}'
    statisticcat_desc = '&statisticcat_desc=' + urllib.parse.quote(stat_desc)
    short_desc = '&short_desc=' + urllib.parse.quote(description)
    location_desc = '&location_desc='+ urllib.parse.quote(state)
    year__GE = '&year__GE=2013'
    format = '&format=CSV'

    parameters = source_desc + sector_desc + commodity_desc + statisticcat_desc + short_desc + location_desc + year__GE + format

    # Retrieve the crop yield data based on criteria defined in the filter list element
    df = get_data(parameters) 

    # Reduce table size and remove elements that are out of project scope
    df = df[(df['year']!=2024)&(df['reference_period_desc']=='YEAR')] 
    df = df[['commodity_desc','year','Value']] 

    # Rename fields to make them easier to understand
    df.rename(columns={'commodity_desc': 'CROP', 'year':'YEAR','Value':'YIELD (Bu/Acre)'}, inplace=True)  

    # Convert year to string to enable join with secondary dataset
    df['YEAR'] = df['YEAR'].astype(str)

    # Concatenate the dataframe along rows with the final crop_yield_df which will contain yield data for all 4 crops
    crop_yield_df = pd.concat([crop_yield_df, df], ignore_index=True)

# Print top 5 records of the final Crop Yield dataframe
crop_yield_df.head() 


,CROP,YEAR,YIELD (Bu/Acre)
0,CORN,2023,201.0
1,CORN,2022,200.0
2,CORN,2021,204.0
3,CORN,2020,177.0
4,CORN,2019,198.0


### Extract, Clean, and Preprocess Climate Data from the NOAA Database

 The climate data is sourced from weather stations operated by the National Oceanic and Atmospheric Administration (NOAA). These weather stations capture daily and monthly records of several climate variables. For this analysis, we will retrieve maximum, minimum, and average monthly temperature (deg F), along with monthly precipitation (mm). For each crop, multiple weather stations per state were selected to approximate climate conditions during growing season. A summary of the growing months is as follows:


 1. Growing months for Corn: June - August
 2. Growing months for Soybean: June - August
 3. Growing months for Barley: May - June
 4. Growing months for Oats: May - June


In [5]:
# Retrieve the climate data from csv files exported from the NOAA Database
def read_climate_data(path, crop, state, growing_months):
    
    df = pd.read_csv(path)

    # Select feature of interest for our analysis
    df = df[["NAME", "DATE", "PRCP", "TAVG", "TMAX", "TMIN"]]

    # Extract State Abbreviation and filter df for state of interest
    df["STATE"] = df["NAME"].str.extract(r",\s*([A-Z]{2})\s")[0]
    df = df.drop(columns=["NAME"])
    df = df[df["STATE"]==state]

    # Date in raw data is in the following format: YYYY-MM. Extract Year and Month
    df["YEAR"] = df["DATE"].str.split("-").str[0]
    df["MONTH"] = df["DATE"].str.split("-").str[1]
    df = df.drop(columns={"DATE"})

    # Filter out any records that are out of project scope
    df = df[(df['YEAR']!='2024')]

    # Map of numerical month value to month
    month_mapping = {
        "01": "January",
        "02": "February",
        "03": "March",
        "04": "April",
        "05": "May",
        "06": "June",
        "07": "July",
        "08": "August",
        "09": "September",
        "10": "October",
        "11": "November",
        "12": "December",
    }

    # Replace numerical month value with month
    df["MONTH"] = df["MONTH"].replace(month_mapping)

    # Filter only for temperature and precipitation during the growing months
    df = df[df['MONTH'].isin(growing_months)].dropna()

    # Convert Precipitation from mm to inches to keep all units in imperial units 
    df['PRCP'] = df['PRCP']*0.0393701

    # Add units to faeture name to enable easy visualization
    df.rename(columns={'PRCP': 'PRCP (in)', 'TMIN': 'TMIN (deg F)','TMAX':'TMAX (deg F)','TAVG':'TAVG (deg F)'}, inplace=True)

    # Create column for corresponding crop type
    df["CROP"] = crop

    return df


In [6]:
# Group by crop and year and aggregate the data to return the mean TAVG, mean TMIN, mean TMAX, and mean PRCP during the growing months
def aggregate_data(climate_df):
    climate_df = climate_df.groupby(['CROP','YEAR']).agg({'PRCP (in)': 'mean', 'TMIN (deg F)': 'mean', 'TMAX (deg F)': 'mean', 'TAVG (deg F)': 'mean'}).round(2).reset_index()

    return climate_df


In [7]:
# Relative paths for .csv data for each crop
corn_paths = "data/Corn Weather Data.csv"
soybean_paths = "data/Soybean Weather Data.csv"
barley_paths = "data/Barley Weather Data.csv"
oats_paths = "data/Oats Weather Data.csv"

# List containing the following tuple for each crop: (crop name, state it is grown, growing months for the crop) 
crops = [
    ("CORN","IA",['June', 'July', 'August']), 
    ("SOYBEANS","IL",['June', 'July', 'August']), 
    ("BARLEY","ID",['May', 'June']), 
    ("OATS","SD",['May', 'June'])
    ]

# List of .csv paths
paths = [corn_paths, soybean_paths, barley_paths, oats_paths]

# For each crop, use read_cimate_data() to create climate dataframes from the downloaded csv files 
# Use the aggregate_data() function to obtain the mean values for the climate variables during the growing months
# Save the modified dataframe for each crop in the climate_data dictionary
climate_data = {}
for path, (crop,state,growing_months) in zip(paths, crops):
    climate_df = read_climate_data(path, crop, state, growing_months)
    climate_data[crop] = aggregate_data(climate_df)

# Print top 5 records of the final climate dataframe for corn
climate_data["CORN"].head() 



,CROP,YEAR,PRCP (in),TMIN (deg F),TMAX (deg F),TAVG (deg F)
0,CORN,2013,0.09,59.57,79.83,69.70
1,CORN,2014,0.21,59.67,77.63,68.67
2,CORN,2015,0.24,59.27,77.67,68.47
3,CORN,2016,0.27,61.20,80.60,70.93
4,CORN,2017,0.11,60.45,82.50,71.45


### Merge the Crop Yield Data with the Climate Data

Now we will combine the crop yield dataframe with the climate dataframes by YEAR and STATE to create our final master dataframe. This dataframe will be used for our exploratory data analysis and visualization. 

In [8]:
final_dfs = []

# For every crop climate dataframe stored in the climate_data dictionary, merge it with the crop yield data and append the result to the final_dfs list
for climate_df in climate_data.values():
    merge_df = pd.merge(crop_yield_df, climate_df, left_on=['CROP', 'YEAR'], right_on=['CROP', 'YEAR'])
    final_dfs.append(merge_df)

# Concatenate the dataframes by rows to create the final master dataframe
final_df = pd.concat(final_dfs, ignore_index=True)

In [9]:
# Print top 5 records of the final dataframe
final_df.head()

,CROP,YEAR,YIELD (Bu/Acre),PRCP (in),TMIN (deg F),TMAX (deg F),TAVG (deg F)
0,CORN,2023,201.0,0.11,59.88,82.18,71.04
1,CORN,2022,200.0,0.12,61.38,83.23,72.33
2,CORN,2021,204.0,0.13,61.45,84.32,72.87
3,CORN,2020,177.0,0.07,62.28,84.42,73.35
4,CORN,2019,198.0,0.15,60.42,79.87,70.13


### Analysis: Descriptive Statistics for crop yield and climate data

We will first describe the yield and climate statistics for each crop. This includes mean, standard deviation, min, and max. This can be used to better understand the variability in yield year-over-year for each crop, as well as the unique climate conditions each crop is exposed to.

In [10]:
# Loop through each crop in crop_data
for crop in final_df['CROP'].unique():
    print(f"Descriptive Statistics for {crop}:")

    # Filter final_df for crop and yield
    crop_df = final_df[(final_df['CROP']==crop)]
    crop_df = crop_df[['YIELD (Bu/Acre)','PRCP (in)', 'TAVG (deg F)', 'TMAX (deg F)', 'TMIN (deg F)']]

    # Print descriptive statistics
    print(crop_df.describe().round(2))
    print('')


Descriptive Statistics for CORN:
       YIELD (Bu/Acre)  PRCP (in)  TAVG (deg F)  TMAX (deg F)  TMIN (deg F)
count            11.00      11.00         11.00         11.00         11.00
mean            192.27       0.17         70.99         81.26         60.71
std              13.29       0.09          1.62          2.37          1.07
min             164.00       0.07         68.47         77.63         59.27
25%             185.00       0.11         69.91         79.85         59.78
50%             198.00       0.13         71.04         81.62         60.45
75%             201.50       0.22         72.13         82.86         61.42
max             204.00       0.36         73.35         84.42         62.28

Descriptive Statistics for SOYBEANS:
       YIELD (Bu/Acre)  PRCP (in)  TAVG (deg F)  TMAX (deg F)  TMIN (deg F)
count            11.00      11.00         11.00         11.00         11.00
mean             58.86       0.16         74.23         83.90         64.56
std              

### Visualization: Plot a line chart for yield and climate variables over time for each crop

We will now create a line chart for each crop to plot the yield and climate trends over time

In [11]:
# List of metrics to loop over
metrics = ['YIELD (Bu/Acre)','PRCP (in)', 'TAVG (deg F)', 'TMAX (deg F)', 'TMIN (deg F)']

# line_chart_list to store each chart that is generated
line_chart_list = []

# Loop over each metric and create a line chart
for metric in metrics:

    line_chart = alt.Chart(final_df).mark_line().encode(
    x=alt.X('YEAR'),
    y=alt.Y(metric),
    color=alt.Color('CROP')
    ).properties(
        title=f'{metric} Trends Over Time',
        width=200,
        height=200 
    )

    line_chart_list.append(line_chart)

# Horizontally concat the charts for each crop
concat_line_charts = alt.hconcat(*line_chart_list)

concat_line_charts


alt.HConcatChart(...)

### Analysis: Determine the correlation between crop yield and the climate variables 

We will now use the Pandas .corr() function to compute spearman correlation of the yield data for each crop and climate variable. This analysis will help us understand the strength of the correlation between yield and the climate variables, providing valuable insight to help answer our project question.

In [12]:
# Empty dictionary to be used to store results for each crop
crop_results = {}

crop_correlations = pd.DataFrame()

# Create the correlation matrix for each crop and store results in crop_results
for crop in final_df['CROP'].unique():

    # Filter final_df for crop of interest
    crop_df = final_df[(final_df['CROP']==crop)] 

    # Compute Spearman correlation coefficient for each variable
    correlation_matrix = crop_df[['YIELD (Bu/Acre)','PRCP (in)', 'TAVG (deg F)', 'TMAX (deg F)', 'TMIN (deg F)']].corr(method='spearman') 
    
    # Store results in crop_results dictionary
    crop_results[crop] = correlation_matrix

# For every crop correlation matrix, extract only the Yield correlation coefficients with the climate variables
for crop, corr_matrix in crop_results.items():

    # Drop the Yield-Yield correlation coefficients 
    yield_corr = corr_matrix['YIELD (Bu/Acre)'].drop('YIELD (Bu/Acre)')  

    # Rename series name to include the specific crop
    yield_corr.name = crop  
    
    # Combine all the yield correlation data into one dataframe
    crop_correlations = pd.concat([crop_correlations, yield_corr], axis=1) 

# Transpose the dataframe
crop_correlations = crop_correlations.T

print(crop_correlations)


          PRCP (in)  TAVG (deg F)  TMAX (deg F)  TMIN (deg F)
CORN       0.232347      0.336364      0.409091      0.290909
SOYBEANS   0.189381      0.707770      0.671240      0.525120
BARLEY     0.508021     -0.209091     -0.372727      0.027273
OATS       0.304160     -0.516151     -0.622146     -0.400026


### Visualization: Plot the correlation coefficients as a heat map

We will now create a heat map of the crop_correlation coefficients

In [13]:
# Transform the schema of the crop_correlations dataframe to make it easier to use when plotting a heatmap
crop_correlations = crop_correlations.reset_index().melt(id_vars='index')
crop_correlations.columns = ['Crop', 'Climate Variable', 'Correlation']

# Create the heatmap using a diverging color scale
heatmap = alt.Chart(crop_correlations).mark_rect().encode(
    x=alt.X('Climate Variable', title='Climate Variable'),
    y=alt.Y('Crop', title='Crop'),
    color=alt.Color(
        'Correlation', 
        scale=alt.Scale(scheme='redblue', domain=[-1, 1]),  # Diverging color scale from -1 to 1
        legend=alt.Legend(title='Spearman Correlation Coefficient')
    )
).properties(
    width=400,
    height=400,
    title='Crop Yield vs Climate Variables Correlation Heatmap'
)

# Create text labels for the heatmap
text = heatmap.mark_text(baseline='middle').encode(
    text=alt.Text('Correlation', format='.2f'), 
    color=alt.value('black')
)

# Combine the heatmap with the text labels
heatmap_with_labels = heatmap + text

heatmap_with_labels.show()

alt.LayerChart(...)

### Visualization: Plot a layered chart for TAVG and Soybean Yield over Time

From the heat map, we see that soybean yield has strong positive correlations with TAVG (0.71). This is the strongest relationship on the heat map. Let's plot both TAVG and yield over time as line charts and overlay the charts to help us better visualize the correlation. 

In [14]:
soybean_data = final_df[final_df['CROP'] == 'SOYBEANS']
soybean_data =soybean_data[['YEAR','YIELD (Bu/Acre)','TAVG (deg F)']]
chart_list = []

# Transform the schema of the soybean_data dataframe to make it easier to use overlaying the charts. This will enable us to add a legend to distinguish the two lines
soybean_data = soybean_data.melt('YEAR', var_name='metric', value_name='value')

# Create a line chart for Yield and TAVG 
for metric in ['YIELD (Bu/Acre)','TAVG (deg F)']:

    linechart_df = soybean_data[soybean_data['metric']==metric]

    # Create the line chart
    chart = alt.Chart(linechart_df).mark_line().encode(
        alt.X('YEAR', title='YEAR'),
        alt.Y('value', 
              title=metric,
              scale=alt.Scale(domain=[48,79])),
        alt.Color('metric', legend=alt.Legend(title="Variable"))  
    ).properties(
        title=f'Soybean Yield and TAVG Trends Over Time',
        width=600,
        height=400   
    )

    chart_list.append(chart)


# Overlay the charts
layered_chart = alt.layer(*chart_list).resolve_scale(
    y='independent'  
)

layered_chart.show()


alt.LayerChart(...)